In [ ]:
conda clean -all

In [3]:
import pandas
import math
import numpy as np

from geneticalgorithm import geneticalgorithm as ga
from qpsolvers import solve_qp

# percentReturnsWeekDataAfterLunc.xlsx
# percentReturnsWeekDataBeforeLunc.xlsx
# percentReturnsWeekData.xlsx
# path = r'data\API binance\percentReturnsMonthDataBeforeLunc.xlsx'
path = r'data\realworld dataset\\Datasets\\DowJones\\DowJones.xlsx'
# Read TestData
my_data = pandas.read_excel(path, header=None)

# Read API Data
my_data = pandas.read_excel(path, index_col=0, header=0)

percentInOut = 0.95
data_in  = my_data.iloc[:round(percentInOut*my_data.shape[0]), :]
data_out = my_data.iloc[round(percentInOut*my_data.shape[0]):, :]
covMatrix = data_in.cov().to_numpy()
corrMatrix = data_in.corr().to_numpy()
expectedVector = np.mean(data_in.to_numpy().to_numpy(), axis=0)

covMatrixOut = data_out.cov().to_numpy()
corrMatrixOut = data_out.corr().to_numpy()
expectedVectorOut = np.mean(my_data, axis=0)

# print("Covariance Matrix is : ")
# print(covMatrix.size)
# print(len(covMatrix))
print(expectedVector)
# print("Correlation Matrix is : ")
# print(corrMatrix)

[0.00425773 0.00194321 0.00257649 0.00197834 0.00236836 0.00296398
 0.00121811 0.00281042 0.00223029 0.00271553 0.00187621 0.00410456
 0.0026708  0.00406927 0.00218431 0.00203078 0.00631501 0.00591688
 0.00240691 0.00212719 0.00421278 0.00224362 0.00289772 0.0031003
 0.00180925 0.00302151 0.00221985]


Solve model 1 using ga

In [2]:
def f(X):
    #     Objective function: Sharpe Ratio
    dim = len(X)
    OF = 0
    pen = 0
    contrant = 0
    xicMa = 0
    for i in range(0, dim):
        contrant += X[i]
        OF += ((X[i])*expectedVector[i])
        for j in range(0, dim):
            xicMa += (X[i])*(X[j])*covMatrix[i, j]
#     Contrant of model by using penalty
    if contrant < 10:
        pen = 500 + 1000*(contrant)
    OF = OF/math.sqrt(xicMa)
    return - OF + pen


def executionGa(varbound, numbersOfAssests,K):
    algorithm_param = {'max_num_iteration': 100000,
                    'population_size': 100,
                    'mutation_probability': 0.2,
                    'elit_ratio': 0.01,
                    'crossover_probability': 0.5,
                    'parents_portion': 0.3,
                    'crossover_type': 'uniform',
                    'max_iteration_without_improv': None}

    # Use for TestData because it have a large number assets, set max_num_iteration to saving time run.
    # model=ga(function=f,dimension=numbersOfAssests,variable_type='bool',variable_boundaries=varbound, algorithm_parameters=algorithm_param)

    # Default for api data
    model = ga(function=f, dimension=numbersOfAssests,
            variable_type='bool', variable_boundaries=varbound)

    model.run()
    outputDict = model.output_dict
    while outputDict['function'] >= (500 + 1000*(10) - 1):
        model.run()
        outputDict = model.output_dict       
    print(f"outputDict: {outputDict}")
    return outputDict

Solve Model 2 (Quadratic programming)

In [3]:
def chosenAssets(arr):
    chosenAssetsList = []
    for n in range(len(arr)):
        if arr[n] == 1:
            chosenAssetsList.append(n)
    return chosenAssetsList

def executionQP(outputDict):
    chosenAssetsList = chosenAssets(outputDict['variable'])
    numberOfAssets = len(chosenAssetsList)
    P = np.zeros((numberOfAssets, numberOfAssets))
    for i in range(numberOfAssets):
        for j in range(numberOfAssets):
            P[i, j] = covMatrix[chosenAssetsList[i] ,chosenAssetsList[j]]                              

    expectedReturns = 0
    q = np.array([0. for i in range(numberOfAssets)])
    A = np.array([1. for i in range(numberOfAssets)])
    
    b = np.array([1.])
    G = -np.array([expectedVector[chosenAssetsList[i]] for i in range(numberOfAssets)])
    h = -np.array([expectedReturns])
    lb = 0.01 * np.ones(numberOfAssets)
    ub = 1 * np.ones(numberOfAssets)
    proportionOfAssetsList = solve_qp(P, q, G=G, h=h, A=A, b=b, lb=lb, ub=ub,solver="cvxopt")
    return chosenAssetsList,  proportionOfAssetsList
    




In [4]:
def runModel(numbersOfAssests, K):
    print(K)
    varbound = np.array([[0, 1]]*numbersOfAssests)

    outputDict = executionGa(varbound, numbersOfAssests, K)
    chosenAssetsList, proportionOfAssetsList = executionQP(outputDict)
    print(f"Proportion Of Assets: x = {proportionOfAssetsList}")
    return chosenAssetsList, proportionOfAssetsList, outputDict

In [5]:
numbersOfAssests = expectedVector.shape[0]
chosenAssetsList, proportionOfAssetsList, outputDict= runModel(numbersOfAssests, 9)
 
  



9


KeyError: 0

In [ ]:
def calculateSharpeRatio(assetsList, proportionOfAssetsList, expectedArr, covMatrix):
    xichMa = 0
    sharpeRatio = 0
    avarageReturn = 0
    dim = len(assetsList) 
    for i in range(0, dim):
        idxAsset1 = assetsList[i]
        avarageReturn += proportionOfAssetsList[i]*expectedArr[idxAsset1]
        for j in range(0, dim):
            idxAsset2 = assetsList[j]
            xichMa += (proportionOfAssetsList[i])*(proportionOfAssetsList[j])*covMatrix[idxAsset1 ,idxAsset2]
    sharpeRatio = avarageReturn/math.sqrt(xichMa)
    return sharpeRatio, avarageReturn, math.sqrt(xichMa)
 
    
testProportionOfAssetsList = [1 for i in range(len(chosenAssetsList))]    

valueOutTest = calculateSharpeRatio(chosenAssetsList, testProportionOfAssetsList, expectedVector, covMatrixOut)
print(f"Sharpe Ratio in sample Out by GA: {valueOutTest[0]}")
valueOut = calculateSharpeRatio(chosenAssetsList, proportionOfAssetsList, expectedVectorOut, covMatrixOut)
# print(valueOut)
print(f"Sharpe Ratio in sample Out: {valueOut[0]}")

print(f"Sharpe Ratio in sample In by GA: {-outputDict['function']}")
valueIn = calculateSharpeRatio(chosenAssetsList, proportionOfAssetsList, expectedVector, covMatrix)
print(f"Sharpe Ratio in sample In: {valueIn[0]}")


# Print name assets if it use api data
if len(covMatrix) == 16:
    nameOfAssets = ['BTC_USD', 'ETH_USD', 'XRP_USD', 'ADA_USD', 'TRX_USD', 'SOL_USD', 'UNI_USD', 'AVAX_USD', 'LINK_USD', 'BNB_USD', 'ATOM_USD', 'ETC_USD', 'NEAR_USD', 'FTM_USD', 'DOGE_USD', 'MATIC_USD']
    print(f"Chosen assets: ")
    for idx in range(len(chosenAssetsList)):
        print(f"{nameOfAssets[chosenAssetsList[idx]]} - {round(proportionOfAssetsList[idx]*100, 2)} %")
elif len(covMatrix) == 17: 
    nameOfAssets = ['BTC_USD', 'ETH_USD', 'XRP_USD', 'ADA_USD', 'TRX_USD', 'SOL_USD', 'UNI_USD', 'AVAX_USD', 'LINK_USD', 'BNB_USD', 'ATOM_USD', 'ETC_USD', 'NEAR_USD', 'LUNC_USD', 'FTM_USD', 'DOGE_USD', 'MATIC_USD']
    print(f"Chosen assets: ")
    for idx in range(len(chosenAssetsList)):
        print(f"{nameOfAssets[chosenAssetsList[idx]]} - {round(proportionOfAssetsList[idx]*100, 2)} %")

Sharpe Ratio in sample Out by GA: nan
Sharpe Ratio in sample Out: 0.4366355965805874
Sharpe Ratio in sample In by GA: nan
Sharpe Ratio in sample In: nan
Choosen assets: 
Asset:2 - 12.5 %
Asset:3 - 12.5 %
Asset:6 - 12.5 %
Asset:7 - 12.5 %
Asset:8 - 12.5 %
Asset:10 - 12.5 %
Asset:12 - 12.5 %
Asset:13 - 12.5 %


Chosen assets: 
XRP_USD - 12.5 %
ADA_USD - 12.5 %
UNI_USD - 12.5 %
AVAX_USD - 12.5 %
LINK_USD - 12.5 %
ATOM_USD - 12.5 %
NEAR_USD - 12.5 %
LUNC_USD - 12.5 %
